In [1]:
import re
import json
import time
import codecs
import pandas as pd
from tqdm import tqdm
from pathlib import Path
setup = True

In [2]:
def clean(text):    
    text = text.lower().strip()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

In [3]:
def cut(temp,doc):

    data = []
    tags ={'item 0': 0, 'item 1': 0, 'item 2': 0, 'item 3': 0, 'item 4': 0, 'item 5': 0, 'item 6': 0, 'item 7': 0, 'item 8': 0, 'item 9': 0, 'item 10': 0, 'item 11': 0, 'item 12': 0, 'item 13': 0, 'item 14': 0, 'item 15': 0, 'item 16': 0}
    for i in temp:
        h = ''.join([j if ord(j) < 128 and ord(j)>31 else ' ' for j in i])
        if len(h)>5 and len(h.split())>1:
            data.append(h)
    
    c = 1
    s = 0
    e = len(data)

    limits = {}
    i = 0
    while i <len(data):
        
        if 'font-size' in data[i] or '/div>' in data[i]:
            e = i
            break
        
        if c<len(tags) and 'item '+str(c) in clean(data[i]):
            e = i
            limits['item '+str(c-1)]=[s,e]
            s = i
            c+=1
        if len(limits)>1 and limits['item 1'][1]-limits['item 1'][0]<10:
            c = 1
            limits = {}
            i+=100
        i+=1
            
        
    limits['item '+str(c-1)]=[s,e]     
    segments = {}
    
    for x in limits:
        
        segments[x]=data[limits[x][0]:limits[x][1]]
        
    for x in segments:
        
        raw = segments[x]
        new = " "
        
        for i in raw:
            if len(i.split())>5:
                new+=i.strip()
                
        segments[x]=new
    #print(limits)
    json.dump(segments,open('items/'+str(doc)+'.json','w'))

In [4]:
if setup:
    count = 0

    for i in tqdm(Path("./text_only").glob("*")):
        try:
            with codecs.open(i,'r', encoding = 'utf-8', errors = 'ignore') as f:
                #print(str(i).split('/')[-1].split('.')[0])
                cut(f.readlines(),str(i).split('/')[-1].split('.')[0])
                count+=1
        except:
            pass

440it [10:32,  1.44s/it]


In [5]:
with open('data/0.txt','r') as file:
    cut(file.readlines(),0)

FileNotFoundError: [Errno 2] No such file or directory: 'data/0.txt'

In [ ]:
x = json.load(open('0.json','r'))
for i in x:
    print("-"*75)
    print(i)
    print(len(x[i].split()))
    print("-"*75)